In [ ]:
!pip install -q --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install -q sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install -q --no-deps unsloth
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
command = (
    "python train.py"
    " ./dataset"
    " --max_steps 50"
    " --learning_rate 4e-5"
    " --max_seq_length 32767"
    " --output_dir beatsaber_model"
)

!{command}

In [ ]:
!rm -r beatsaber_model

In [ ]:
!python infer.py "./beatsaber_model" "desert fury.ogg" --bpm 172 --duration 30 --temperature 2.0 --min_p 0.1

In [ ]:
from infer import load_model, prepare_audio
from data import (
    BOS, EOS, AUDIO_START, AUDIO_END, NOTES_START,
    TIME_RANGE, NOTE_RANGE, AUDIO_RANGE, audio_tokens_to_sequence,
    time_to_token, decode_note, get_audio_tokenizer
)
import torch
import itertools

AUDIO_TOKENIZER = get_audio_tokenizer(bandwidth=1.5)
model, tokenizer = load_model("./beatsaber_model")
audio = prepare_audio("desert fury.ogg", 30)

audio_tokens = AUDIO_TOKENIZER.encode(audio)

flattened_audio_tokens = list(itertools.chain.from_iterable(audio_tokens))
print(f"{tokenizer.decode(flattened_audio_tokens)}")

In [ ]:
from maps import ZippedBeatSaberDataset
from data import CachedDataset, create_training_sequence
from pathlib import Path
from infer import load_model

dataset_dir = Path("dataset")
zip_files = list(dataset_dir.glob("*.zip"))
raw_dataset = ZippedBeatSaberDataset(dataset_dir)
cached_dataset = CachedDataset(raw_dataset)

audio_tokens, notes, bpm = cached_dataset[0]

sequence = create_training_sequence(audio_tokens, notes, max_length=32767)

model, tokenizer = load_model("./beatsaber_model")

decoded_text = tokenizer.decode(sequence)
print(f"{decoded_text}")

In [ ]:
model = None